In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np

x_validation_data_path = [
                       "/content/drive/My Drive/_SPL3/model/x_hockey_validation.pickle",
                       "/content/drive/My Drive/_SPL3/model/x_movie_validation.pickle",
                       "/content/drive/My Drive/_SPL3/model/x_crowd_validation.pickle"
                    ]
y_validation_data_path = [
                       "/content/drive/My Drive/_SPL3/model/y_hockey_validation.pickle",
                       "/content/drive/My Drive/_SPL3/model/y_movie_validation.pickle",
                       "/content/drive/My Drive/_SPL3/model/y_crowd_validation.pickle"
                    ]
x_data_path = [
                       "/content/drive/My Drive/_SPL3/model/x_hockey.pickle",
                       "/content/drive/My Drive/_SPL3/model/x_movie.pickle",
                       "/content/drive/My Drive/_SPL3/model/x_crowd.pickle"
                    ]
y_data_path = [
                       "/content/drive/My Drive/_SPL3/model/y_hockey.pickle",
                       "/content/drive/My Drive/_SPL3/model/y_movie.pickle",
                       "/content/drive/My Drive/_SPL3/model/y_crowd.pickle"
                    ]

# loading saved dataset with pickle
x_validation, y_validation, x, y = None, None, None, None
for i in range(0, len(x_data_path)):
    pickle_input_x_validation = open(x_validation_data_path[i], "rb")
    # x_validation = pickle.load(pickle_input_x_validation)
    new_x_valid = pickle.load(pickle_input_x_validation)
    pickle_input_x_validation.close()

    pickle_input_y_validation = open(y_validation_data_path[i], "rb")
    new_y_valid = pickle.load(pickle_input_y_validation)
    pickle_input_y_validation.close()

    pickle_input_x = open(x_data_path[i], "rb")
    new_x = pickle.load(pickle_input_x)
    pickle_input_x.close()

    pickle_input_y = open(y_data_path[i], "rb")
    new_y = pickle.load(pickle_input_y)
    pickle_input_y.close()

    if i == 0:
        x_validation = new_x_valid
        y_validation = new_y_valid
        x = new_x
        y = new_y
    else:
        x_validation = np.concatenate( (x_validation, new_x_valid) )
        y_validation = np.concatenate( (y_validation, new_y_valid) )
        x = np.concatenate( (x, new_x) )
        y = np.concatenate( (y, new_y) )
    print("Training Set: New=", new_x.shape, "\tMerged=", x.shape)
    print("Validation Set: New=", new_x_valid.shape, "\tMerged=", x_validation.shape, "\n")


x_validation = x_validation.astype(dtype=np.float32)/255
y_validation = np.array(y_validation)
x = x.astype(dtype=np.float32)/255
y = np.array(y)

# Taking Dump of The Complete Merged Dataset
pickle_write_x = open("/content/drive/My Drive/_SPL3/model/x_validation_all.pickle", "wb")
pickle.dump(x_validation, pickle_write_x)
pickle_write_x.close()

pickle_write_y = open("/content/drive/My Drive/_SPL3/model/y_validation_all.pickle", "wb")
pickle.dump(y_validation, pickle_write_y)
pickle_write_y.close()

pickle_write_x = open("/content/drive/My Drive/_SPL3/model/x_all.pickle", "wb")
pickle.dump(x, pickle_write_x)
pickle_write_x.close()

pickle_write_y = open("/content/drive/My Drive/_SPL3/model/y_all.pickle", "wb")
pickle.dump(y, pickle_write_y)
pickle_write_y.close()


Training Set: New= (1810, 10, 100, 100, 3) 	Merged= (1810, 10, 100, 100, 3)
Validation Set: New= (204, 10, 100, 100, 3) 	Merged= (204, 10, 100, 100, 3) 

Training Set: New= (435, 10, 100, 100, 3) 	Merged= (2245, 10, 100, 100, 3)
Validation Set: New= (48, 10, 100, 100, 3) 	Merged= (252, 10, 100, 100, 3) 

Training Set: New= (983, 10, 100, 100, 3) 	Merged= (3228, 10, 100, 100, 3)
Validation Set: New= (112, 10, 100, 100, 3) 	Merged= (364, 10, 100, 100, 3) 



In [ ]:
import numpy as np
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed,Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LSTM,Bidirectional
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard

K.set_image_data_format('channels_last')
NAME = "VIOLENCE-DETECTOR"
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

## training the CNN
cnn = Sequential()
#input
cnn.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#1st layer
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
#converting to 1-d tensor
cnn.add(Flatten())

model=Sequential()
model.add(TimeDistributed(cnn,input_shape=x.shape[1:]))
model.add(Bidirectional(LSTM(32)))
#model.add(LSTM(32))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
#model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
print(model.summary())
model.fit(x, y, epochs=25, validation_data=(x_validation, y_validation), batch_size=5, callbacks=[tensorboard])
model.save("ViolenceDetection.h5")